In [10]:
import sys
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "2"

import jax
import jax.tree_util as jtu
import jax.scipy as jsp
import jax.numpy as jnp

from functools import partial
from scipy.io import savemat
from itertools import product

import optax
import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler

import matplotlib.pyplot as plt

from base import loss_fn, predict_fn, get_params
from utils import train_fn, latexify
from data import generate_heinonen_gp_data, get_nonstat2d

jax.config.update("jax_enable_x64", True)

%reload_ext watermark

In [15]:
method = "delta_inducing"

aq_df = pd.read_pickle(f"results/aq_results_{method}.pkl")

rmse_df = pd.DataFrame(index=product([1, 0], repeat=3), columns=range(3))
nlpd_df = pd.DataFrame(index=product([1, 0], repeat=3), columns=range(3))
# rmse

for col in rmse_df.columns:
    for index in rmse_df.index:
        y_test = aq_df[index][col]["y_test"]
        pred_scale = jnp.sqrt(aq_df[index][col]["pred_var"] + aq_df[index][col]["pred_omega"]**2)
        pred_mean = aq_df[index][col]["pred_mean"]
        rmse_df[col][index] = jnp.sqrt(jnp.mean((y_test - pred_mean)**2 ))
        nlpd_df[col][index] = -jsp.stats.norm.logpdf(y_test, pred_mean, pred_scale).mean()
        
rmse_df["mean"] = rmse_df.mean(axis=1)
nlpd_df["mean"] = nlpd_df.mean(axis=1)

display(rmse_df.style.highlight_min(color='green'))
display(nlpd_df.style.highlight_min(color='green'))

,0,1,2,mean
"(1, 1, 1)",0.13631780715654562,0.12410173512812761,0.17092846355544816,0.143783
"(1, 1, 0)",0.11582523957919166,0.1264983642065729,0.195851777937958,0.146058
"(1, 0, 1)",0.13426227707229052,0.12729957688011573,0.168187425424792,0.143250
"(1, 0, 0)",0.12304426307430011,0.12008191648905642,0.17309596976120428,0.138741
"(0, 1, 1)",0.09537358951539636,0.11556788078819918,0.1434255542868254,0.118122
"(0, 1, 0)",0.10876860149709543,0.14821807740505208,0.17597911136688044,0.144322
"(0, 0, 1)",0.09634582885566205,0.11527361821446727,0.1405753832345957,0.117398
"(0, 0, 0)",0.10100032517372289,0.12982736869487332,0.19984544541949706,0.143558


,0,1,2,mean
"(1, 1, 1)",-0.0026317473282119865,-0.2925869426432193,0.46858512632216615,0.057789
"(1, 1, 0)",-0.16218051882739418,-0.2959484840459032,1.112699764852732,0.218190
"(1, 0, 1)",-0.09694611028264913,-0.36616751535210146,0.43763252375500833,-0.008494
"(1, 0, 0)",-0.27897207030693144,-0.6161742890258779,0.48260949774586975,-0.137512
"(0, 1, 1)",0.9659903705835804,-0.4528155839748129,-0.18938948403716005,0.107928
"(0, 1, 0)",-0.31350311360915817,-0.3165295572392638,-0.18660634436013396,-0.272213
"(0, 0, 1)",-0.4653478524048207,-0.5959655440610576,-0.18157822397080028,-0.414297
"(0, 0, 0)",-0.3221937088255261,-0.5609847941577193,-0.15746286561227157,-0.346880
